# Example 3: Extended usage and output diagnostics

In this notebook, we demonstrate extended usage of `pyvbmc`. We will:
- take a brief look at `pyvbmc`'s diagnostic output,
- show you how to continue optimization starting from the results of a previous run, and
- show you how to save the results of optimization to disk.

This notebook is Part 3 of a series of notebooks in which we present various example usages for VBMC with the `pyvbmc` package.

In [1]:
import numpy as np
import scipy.stats as scs
from scipy.optimize import minimize
from pyvbmc.vbmc import VBMC
import dill

## 1. Convergence Diagnostics

For demonstration purposes, we will run VBMC with a restricted budget of function evaluations, insufficient to achieve convergence. Then we will inspect the output diagnostics, and resume optimization.

We use a higher-dimensional analogue of the same toy target function in Example 1, a broad [Rosenbrock's banana function](https://en.wikipedia.org/wiki/Rosenbrock_function) in $D = 4$.

In [2]:
D = 4  # A four-dimensional problem
prior_mu = np.zeros(D)
prior_var = 3 * np.ones(D)


def log_prior(theta):
    """Multivariate normal prior on theta."""
    cov = np.diag(prior_var)
    return scs.multivariate_normal(prior_mu, cov).logpdf(theta)

The likelihood function of your model will in general depend on the observed data. This data can be fixed as a global variable, as we did directly above for `prior_mu` and `prior_var`. It can also be defined by a default second argument: to `pyvbmc` there is no difference so long as the function can be called with only a single argument (the parameters `theta`):

In [3]:
def log_likelihood(theta, data=np.ones(D)):
    """D-dimensional Rosenbrock's banana function."""
    # In this simple demo the data just translates the parameters:
    theta = np.atleast_2d(theta)
    theta = theta + data

    x, y = theta[:, :-1], theta[:, 1:]
    return -np.sum((x**2 - y) ** 2 + (x - 1) ** 2 / 100, axis=1)


def log_joint(theta, data=np.ones(D)):
    return log_likelihood(theta, data) + log_prior(theta)

In [4]:
LB = np.full(D, -np.inf)  # Lower bounds
UB = np.full(D, np.inf)  # Upper bounds
PLB = np.full(D, prior_mu - np.sqrt(prior_var))  # Plausible lower bounds
PUB = np.full(D, prior_mu + np.sqrt(prior_var))  # Plausible upper bounds

In a typical inference scenario, we recommend starting from a "good" point (i.e. one near the mode). We can run a  quick preliminary optimization (though a more extensive optimization would not harm).

In [5]:
np.random.seed(42)
x0 = np.random.uniform(PLB, PUB)  # Random point inside plausible box
x0 = minimize(lambda t: -log_joint(t), x0).x

In [6]:
# Limit number of function evaluations
options = {
    "max_fun_evals": 10 * D,
}
# We can specify either the log-joint, or the log-likelihood and log-prior.
# In other words, the following lines are equivalent:
vbmc = VBMC(
    log_likelihood,
    x0,
    LB,
    UB,
    PLB,
    PUB,
    user_options=options,
    log_prior=log_prior,
)
# vbmc = VBMC(
#     log_joint,
#     x0, LB, UB, PLB, PUB, user_options=options,
# )

Reshaping x0 to row vector.
Reshaping lower bounds to (1, 4).
Reshaping upper bounds to (1, 4).
Reshaping plausible lower bounds to (1, 4).
Reshaping plausible upper bounds to (1, 4).


(`pyvbmc` expects the bounds to be `(1, D)` row vectors, and the initial point(s) to be of shape `(n, D)`, but it will accept and re-shape vectors of shape `(D,)` as well.)

In [7]:
vp, elbo, elbo_sd, success_flag, info = vbmc.optimize()

Beginning variational optimization assuming EXACT observations of the log-joint.
 Iteration  f-count    Mean[ELBO]    Std[ELBO]    sKL-iter[q]   K[q]  Convergence  Action
     0         10          -5.13         3.81     98089.49        2        inf     start warm-up
     1         15          -4.91         1.38         2.14        2        inf     
     2         20          -3.21         1.48         1.80        2       40.7     
     3         25          -4.58         1.27         1.45        2         33     
     4         30          -5.33         0.30         0.83        2       17.3     
     5         35          -4.86         1.31         1.90        2       37.6     
     6         40          -4.61         0.76         1.63        2       30.6     
   inf         40          -4.34         0.75         1.64       50       30.6     finalize
Inference terminated: reached maximum number of function evaluations options.max_fun_evals.
Estimated ELBO: -4.335 +/-0.749.
Caution: Re

`pyvbmc` is warning us that convergence is doubtful. We can look at the output for more information and diagnostics.

In [8]:
success_flag

False

`False` means that `pyvbmc` has not converged to a stable solution within the given number of function evaluations.

In [9]:
info

{'function': '<function VBMC.__init__.<locals>.log_joint at 0x7f0bb9b81ee0>',
 'problemtype': 'unconstrained',
 'iterations': 6,
 'funccount': 40,
 'bestiter': 6,
 'trainsetsize': 40,
 'components': 50,
 'rindex': 30.586103804330275,
 'convergencestatus': 'no',
 'overhead': nan,
 'rngstate': 'rng',
 'algorithm': 'Variational Bayesian Monte Carlo',
 'version': '0.0.1',
 'message': 'Inference terminated: reached maximum number of function evaluations options.max_fun_evals.',
 'elbo': -4.335372122828826,
 'elbo_sd': 0.7491560646729979}

In the `info` dictionary:
- the `convergencestatus` field says 'no' (probable lack of convergence);
- the reliability index `rindex` is 3.68, (should be less than 1).
Our diagnostics tell us that this run has not converged, suggesting to increase the budget.

Note that convergence to a solution does not mean that it is a _good_ solution. You should always check the returned variational posteriors, and ideally should compare across multiple runs of `pyvbmc`.

## Continuing optimization from a previous result

We can continue running `pyvbmc` where we left off by calling the `optimize()` method again. But first we will change the function evaluation budget to a more realistic value.

In [10]:
# This verbose syntax is required to avoid unintentionally modifying
# options after initialization:
vbmc.options.__setitem__("max_fun_evals", 50 * (D + 2), force=True)
vbmc.optimize()

Beginning variational optimization assuming EXACT observations of the log-joint.
 Iteration  f-count    Mean[ELBO]    Std[ELBO]    sKL-iter[q]   K[q]  Convergence  Action
     0         50          -6.16         1.85         0.60        2        inf     
     1         55          -4.76         0.09         0.80        2        inf     
     2         60          -4.66         0.03         0.05        2       1.35     
     3         65          -4.58         0.01         0.03        3      0.762     
     4         70          -4.55         0.01         0.01        4      0.303     
     5         75          -4.51         0.01         0.01        5      0.325     
     6         75          -4.20         0.16         0.55        6       10.7     rotoscale
     7         80          -4.34         0.03         0.12        6       2.63     
     8         85          -4.34         0.02         0.01        7      0.195     
     9         90          -4.33         0.01         0.00      

(VariationalPosterior:
     self.D = 4,
     self.K = 50,
     self.mu = (4, 50) ndarray,
     self.w = (1, 50) ndarray,
     self.sigma = (1, 50) ndarray,
     self.lambd = 
         [[1.1971],
          [0.9322],
          [0.8981],
          [0.9442]] : ndarray,
     self.stats = <dict object at 0x7f0bb59b6f80>,
     self._mode = None,
     self.bounds = None,
     self.delta = [[0., 0., 0., 0.]] : ndarray,
     self.eta = (1, 50) ndarray,
     self.gp = GP:
         self.D = 4,
         self.covariance = <gpyreg.covariance_functions.SquaredExponential object at 0x7f0bb5a978e0>,
         self.mean = <gpyreg.mean_functions.NegativeQuadratic object at 0x7f0bb5a974c0>,
         self.noise = <gpyreg.noise_functions.GaussianNoise object at 0x7f0bb5a97b20>,
         self.X = (115, 4) ndarray,
         self.y = (115, 1) ndarray,
         self.s2 = None,
         self.lower_bounds = (15,) ndarray,
         self.upper_bounds = (15,) ndarray,
         self.posteriors = 
              <gpyreg.

## 3. Saving results

We can also save the `VBMC` instance to disk and reload it later, in order to continue optimization, sample from the posterior, check the results, etc.

In [11]:
with open("vbmc_test_save.pkl", "wb") as f:
    dill.dump(vbmc, f)

In [12]:
with open("vbmc_test_save.pkl", "rb") as f:
    vbmc_2 = dill.load(f)
vbmc_2.vp.sample(5)

(array([[-1.24485978, -0.4021069 ,  0.01427758,  1.00368226],
        [-0.07017428, -0.06677729,  0.2468989 ,  0.26978397],
        [-2.09081822,  0.4115007 ,  0.81232483,  2.03394574],
        [-0.48901898,  0.20464576, -0.06871874, -0.18385511],
        [-0.56658237, -0.77832803, -0.93870159, -0.06641562]]),
 array([13,  3, 31,  0,  2]))

## 5. Conclusions

In this notebook, we have given a brief overview of `pyvbmc`'s output diagnostics, and shown how to save, load, and resume optimization. 

In the next notebook, we will illustrate running `pyvbmc` multiple times in order to validate the results.

## Example 3: full code

The following cell includes in a single place all the code used in Example 3, without the extra fluff.

In [13]:
assert False  # skip this cell

import numpy as np
import scipy.stats as scs
from scipy.optimize import minimize
from pyvbmc.vbmc import VBMC
import dill


D = 4  # A four-dimensional problem
prior_mu = np.zeros(D)
prior_var = 3 * np.ones(D)


def log_prior(theta):
    """Multivariate normal prior on theta."""
    cov = np.diag(prior_var)
    return scs.multivariate_normal(prior_mu, cov).logpdf(theta)


def log_likelihood(theta, data=np.ones(D)):
    """D-dimensional Rosenbrock's banana function."""
    # In this simple demo the data just translates the parameters:
    theta = np.atleast_2d(theta)
    theta = theta + data

    x, y = theta[:, :-1], theta[:, 1:]
    return -np.sum((x**2 - y) ** 2 + (x - 1) ** 2 / 100, axis=1)


def log_joint(theta, data=np.ones(D)):
    return log_likelihood(theta, data) + log_prior(theta)


LB = np.full(D, -np.inf)  # Lower bounds
UB = np.full(D, np.inf)  # Upper bounds
PLB = np.full(D, prior_mu - np.sqrt(prior_var))  # Plausible lower bounds
PUB = np.full(D, prior_mu + np.sqrt(prior_var))  # Plausible upper bounds


np.random.seed(42)
x0 = np.random.uniform(PLB, PUB)  # Random point inside plausible box
x0 = minimize(lambda t: -log_joint(t), x0).x


# Limit number of function evaluations
options = {
    "max_fun_evals": 10 * D,
}
# We can specify either the log-joint, or the log-likelihood and log-prior.
# In other words, the following lines are equivalent:
vbmc = VBMC(
    log_likelihood,
    x0,
    LB,
    UB,
    PLB,
    PUB,
    user_options=options,
    log_prior=log_prior,
)
# vbmc = VBMC(
#     log_joint,
#     x0, LB, UB, PLB, PUB, user_options=options,
# )


vp, elbo, elbo_sd, success_flag, info = vbmc.optimize()


success_flag


info


# This verbose syntax is required to avoid unintentionally modifying
# options after initialization:
vbmc.options.__setitem__("max_fun_evals", 50 * (D + 2), force=True)
vbmc.optimize()


with open("vbmc_test_save.pkl", "wb") as f:
    dill.dump(vbmc, f)


with open("vbmc_test_save.pkl", "rb") as f:
    vbmc_2 = dill.load(f)
vbmc_2.vp.sample(5)

AssertionError: 

## Acknowledgments

Work on the `pyvbmc` package was funded by the [Finnish Center for Artificial Intelligence FCAI](https://fcai.fi/).